# Simulate AWS DMS output

In my thesis, the input files are pretended to be written by AWS DMS. Instead of actually creating a relational database and extracting it using AWS DMS, I have chosen to directly create the output files.

In [10]:
import pandas as pd
import random
import os
from helpers.paths import PathMerger
from faker import Faker
from datetime import datetime, timedelta, date, timezone
from dateutil import tz
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

fake = Faker()

## Functions

In [11]:
def generate_customers(first_registration:datetime):
    customers = []
    
    # Memorized value for the creation time of the previous customer.
    created_prev = None
    
    # A single hard-coded customer for reference with an id of 0
    customer = {
        'id': 0, 
        'username': 'janisourander@kamk.fi',
        'password': fake.md5(),
        'created': datetime(1970, 1, 15, 10, 0, 0),
        'modified': datetime(1970, 2, 20, 12, 34, 56),
    }
    
    customers.append(customer)

    for i in range(9):
        
        if created_prev is None:
            # The first ever customer was created at 13:37, 15th of Jan 2020.
            created = first_registration
        else:
            # Each other customer were created n second later than the previous.
            created = created_prev + timedelta(seconds=random.randrange(1, 3600))
        
        customer = {
            'id': len(customers), 
            'username': fake.unique.ascii_email(),
            'password': fake.md5(),
            'created': created,
            'modified': created + timedelta(seconds=random.randrange(0, 3600 * 24 * 365)),
        }
        
        customers.append(customer)
        
        # Set current created datetime as previous
        created_prev = created

    return pd.DataFrame.from_records(customers)
    
    
def generate_device_models():
    devices = [
        {
            'id': 0, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Red',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 1),
            'modified': datetime(2010, 3, 21, 12, 0, 1),
        },
        {
            'id': 1, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Black',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 2),
            'modified': datetime(2010, 3, 21, 12, 0, 2),
        },
        {
            'id': 2, 
            'release_date': date(2010, 11, 1),
            'name': 'Super Gadget 100',
            'color': 'Pink',
            'description': 'lorem ipsum',
            'created': datetime(2010, 8, 5, 7, 0, 0),
            'modified': datetime(2010, 8, 5, 7, 0, 0),
        },
        {
            'id': 3, 
            'release_date': date(2018, 5, 13),
            'name': 'Super Gadget 200',
            'color': 'White',
            'description': 'lorem ipsum',
            'created': datetime(2018, 3, 20, 12, 1, 1),
            'modified': datetime(2018, 3, 20, 12, 1, 1),
        },
    ]
    
    return pd.DataFrame.from_records(devices)


def generate_devices(customers: pd.DataFrame, devices: pd.DataFrame):

    # List of unique model numbers
    unique_device_models = list(devices.id.unique())
    
    # Container
    devices = []
    i = 0

    for (_, cust) in customers.iterrows():
        
        # Fetch the creation date of user and add some
        created = cust.created + timedelta(seconds=random.randrange(1, 3600))
        
        # Create 1 or 2 devices per user by random
        for x in range(random.randrange(1, 3)):
            device = {
                'id': len(devices), 
                'customer_fk': cust.id,
                'model_fk': random.choice(unique_device_models),
                'serial_number': fake.ssn(),
                'created': created,
                'modified': created + timedelta(seconds=random.randrange(1, 3600)),
            }

            devices.append(device)
    
    return pd.DataFrame.from_records(devices)

        
def generate_customer_details(customers: pd.DataFrame):
    
    # Container
    details = []
    
    for (_, cust) in customers.iterrows():
        detail = {
            'id': len(details),
            'customer_fk': cust.id,
            'birthday': fake.date_of_birth(),
            'language': fake.language_code(),
            'street_address': fake.street_address(),
            'postal_code': fake.postalcode(),
            'city': fake.city(),
            'country': fake.country_code(),
            'phone_number': fake.phone_number(),
            'created': cust.created,
            'modified': cust.created,
        }
        
        details.append(detail)
        
    return pd.DataFrame.from_records(details)

## Create Directories

In [12]:
# Mapping
table_paths = {}

# Map of combinations to create: (db, table)
tables = [
    ('customers', 'customers'), 
    ('customers', 'customer_details'),
    ('devices', 'device_models'),
    ('devices', 'devices')
]

for db, table in tables:
    dt = f'{db}.{table}'
    sp = PathMerger(db, table).staging
    dp = os.path.join(sp, 'LOAD00000001.parquet')
    
    # Create dirs
    if not os.path.exists(sp):
        os.makedirs(sp)
    
    # Append
    table_paths[dt] = dp

## Generate Datasets

### Customers

In [13]:
# Generate
df_c = generate_customers(first_registration=datetime(2020, 1, 15, 13, 37, 0))

# Write
df_c.to_parquet(table_paths['customers.customers'])

# Show
df_c

,id,username,password,created,modified
0,0,janisourander@kamk.fi,873619955484beec8f72b34ae8afa995,1970-01-15 10:00:00,1970-02-20 12:34:56
1,1,meadowsmary@hotmail.com,584d1b0045918409f3e2d37b955fe86e,2020-01-15 13:37:00,2020-11-04 01:27:47
2,2,nicolevelasquez@wells.com,9154f32482ea97480b58dc53bbd401d6,2020-01-15 14:30:49,2020-07-20 09:50:47
3,3,brownamber@liu.com,66767e428490d1b3f911830241b6f4d4,2020-01-15 15:07:30,2020-11-04 10:08:36
4,4,jennifer81@gmail.com,2b302950516a5a022ddb3932be3ad8e9,2020-01-15 16:01:48,2020-10-05 13:01:52
5,5,richard48@rodriguez.info,07066cf0cff68b165211648102f46b18,2020-01-15 16:27:24,2020-10-27 19:32:08
6,6,maria19@gmail.com,4e059cb3b1bede1b7d8077c422541678,2020-01-15 17:18:51,2020-11-28 18:50:21
7,7,alexanderpalmer@miles-briggs.biz,c1b82c28099c44884230cfae5233cf91,2020-01-15 17:42:46,2020-11-17 17:56:10
8,8,antonioshaw@hatfield.com,426aecbf939872f0b2ac67201a35452c,2020-01-15 18:26:36,2020-11-08 14:26:26
9,9,christina03@payne.net,01077914a3eb23a3ae2ce51d8063330d,2020-01-15 19:11:43,2020-08-05 21:24:51


### Device Models

In [14]:
# Generate
df_dm = generate_device_models()

# Write
df_dm.to_parquet(table_paths['devices.device_models'])

# Show
df_dm

,id,release_date,name,color,description,created,modified
0,0,2010-05-15,Super Gadget 100,Red,lorem ipsum,2010-03-21 12:00:01,2010-03-21 12:00:01
1,1,2010-05-15,Super Gadget 100,Black,lorem ipsum,2010-03-21 12:00:02,2010-03-21 12:00:02
2,2,2010-11-01,Super Gadget 100,Pink,lorem ipsum,2010-08-05 07:00:00,2010-08-05 07:00:00
3,3,2018-05-13,Super Gadget 200,White,lorem ipsum,2018-03-20 12:01:01,2018-03-20 12:01:01


### Devices

In [15]:
# Generate devices based on customers and device models
df_d = generate_devices(df_c, df_dm)

# Write
df_dm.to_parquet(table_paths['devices.devices'])

# Show
df_d

,id,customer_fk,model_fk,serial_number,created,modified
0,0,0,0,250-87-6499,1970-01-15 10:19:29,1970-01-15 10:41:42
1,1,0,1,313-13-2641,1970-01-15 10:19:29,1970-01-15 10:29:06
2,2,1,1,023-97-9652,2020-01-15 14:13:32,2020-01-15 14:47:49
3,3,2,0,138-09-3405,2020-01-15 15:12:17,2020-01-15 15:55:55
4,4,3,2,372-13-9651,2020-01-15 15:46:29,2020-01-15 16:40:55
5,5,4,1,679-56-5253,2020-01-15 16:18:22,2020-01-15 16:19:07
6,6,4,2,204-90-5406,2020-01-15 16:18:22,2020-01-15 16:38:09
7,7,5,2,898-51-6891,2020-01-15 17:23:25,2020-01-15 17:41:52
8,8,5,3,265-90-0293,2020-01-15 17:23:25,2020-01-15 17:46:42
9,9,6,2,755-04-3131,2020-01-15 17:22:42,2020-01-15 18:08:47


### Customer_details

In [16]:
# Generate customer details. One per customer.
df_cd = generate_customer_details(df_c)

# Write
df_dm.to_parquet(table_paths['customers.customer_details'])

df_cd

,id,customer_fk,birthday,language,street_address,postal_code,city,country,phone_number,created,modified
0,0,0,1930-06-19,ast,7371 Benson Drive,29310,Hillview,TT,(733)648-0923x50217,1970-01-15 10:00:00,1970-01-15 10:00:00
1,1,1,1939-07-11,lzh,21429 Nicole Isle,78802,Washingtontown,NI,653-022-6594,2020-01-15 13:37:00,2020-01-15 13:37:00
2,2,2,1951-08-26,se,46125 Rush Coves,78111,Lake Timothyton,AU,529-922-8718x4904,2020-01-15 14:30:49,2020-01-15 14:30:49
3,3,3,1967-03-27,ce,024 Jennifer Freeway Apt. 428,85182,Robertsfort,AT,172-139-1020,2020-01-15 15:07:30,2020-01-15 15:07:30
4,4,4,2019-05-15,sd,0962 Christian Lights Suite 070,55085,Raymouth,HN,378-714-1433,2020-01-15 16:01:48,2020-01-15 16:01:48
5,5,5,1996-02-13,ti,27059 Martinez Turnpike Suite 298,63813,North Deborah,MY,4981493174,2020-01-15 16:27:24,2020-01-15 16:27:24
6,6,6,1977-12-13,bn,268 Ian Club Suite 519,15639,Markfort,AF,001-824-750-8920x37601,2020-01-15 17:18:51,2020-01-15 17:18:51
7,7,7,1991-11-24,tr,84399 Parks Radial Apt. 796,89693,West Claire,LY,(651)927-7839x05817,2020-01-15 17:42:46,2020-01-15 17:42:46
8,8,8,2008-06-16,st,5313 Morgan Spring,08831,Alexanderberg,BT,955-889-9316,2020-01-15 18:26:36,2020-01-15 18:26:36
9,9,9,1931-01-17,mk,9419 Chapman Forge,82814,West Debra,KM,456-912-9135,2020-01-15 19:11:43,2020-01-15 19:11:43


## List resulting files

In [17]:
# Maximum lenght of a path
max_w = max([len(x[1]) for x in table_paths.items()])

# UNIX epoch start
epoch = datetime(1970, 1, 1, tzinfo=timezone.utc)

for k, f in table_paths.items():

    s = os.stat(f)
    f_size = s.st_size // 1024 
    f_mod = (
        (epoch + timedelta(seconds=s.st_mtime))
        .astimezone(tz.gettz('Europe/Helsinki'))
        .strftime('%y-%m-%d %H:%M:%S')
    )
    
    print(f"{f_mod} | {f:<{max_w}} | {f_size:>2d} KB")

21-08-04 14:21:50 | S3\staging\dms\company_rds\customers\customers\LOAD00000001.parquet        |  5 KB
21-08-04 14:21:50 | S3\staging\dms\company_rds\customers\customer_details\LOAD00000001.parquet |  5 KB
21-08-04 14:21:50 | S3\staging\dms\company_rds\devices\device_models\LOAD00000001.parquet      |  5 KB
21-08-04 14:21:50 | S3\staging\dms\company_rds\devices\devices\LOAD00000001.parquet            |  5 KB
